In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
# Define Conv Module
def conv_module(x, filters, kernel_size, stride):
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

In [3]:
# Define Inception Module
def inception_module(x, ch1_filters, ch3_filters):
    branch1x1 = layers.Conv2D(ch1_filters, (1, 1), padding='same')(x)

    branch3x3 = layers.Conv2D(ch3_filters, (3, 3), padding='same')(x)

    x = layers.concatenate([branch1x1, branch3x3], axis=-1)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

In [4]:
# Define Downsample Module
def downsample_module(x, filters, stride):
    x = layers.Conv2D(filters, (3, 3), strides=stride, padding='same')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

In [5]:
input_height = 128
input_width = 128
input_channels = 3 

num_modules = 3
conv_filters = 64
conv_kernel_size = (3, 3)
conv_stride = (1, 1)

inception_ch1_filters = 32
inception_ch3_filters = 32

downsample_filters = 128
downsample_stride = (2, 2)

output_channels = 10 

# Input layer
input_layer = layers.Input(shape=(input_height, input_width, input_channels))
x = input_layer

In [6]:
# Build the architecture
for _ in range(num_modules):  # Specify the desired number of modules
    # Conv Module
    x_conv = conv_module(x, conv_filters, conv_kernel_size, conv_stride)

    # Inception Module
    x_inception = inception_module(x_conv, inception_ch1_filters, inception_ch3_filters)

    # Downsample Module
    x_downsample = downsample_module(x_conv, downsample_filters, downsample_stride)

    # Concatenate outputs
    x = layers.concatenate([x_inception, x_downsample], axis=-1)

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 128, 128, 64), (None, 32, 32, 128)]

In [ ]:
# Additional Conv Module after concatenation
x = conv_module(x, conv_filters, conv_kernel_size, conv_stride)

# Additional Batch Norm layer
x = layers.BatchNormalization()(x)

# Additional ReLU activation
x = layers.Activation('relu')(x)

In [ ]:
# Output layer (adjust filters and kernel_size based on your task)
output_layer = layers.Conv2D(output_channels, (1, 1), padding='same', activation='softmax')(x)

In [ ]:
# Create the model
model = models.Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()